# Simple Generative QA System
A generative QA system for documents using RAG approach.

In [32]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [33]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [34]:
from datasets import load_dataset
from haystack import Document

# Load a pre-processed sevent wonders of the ancient world
#  
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc['content'], meta=doc['meta']) for doc in dataset]

## Initialize a Document Embedder
SentenceTransformersDocumentEmbedder is used to embed a list of documet using transofmer models that supports it.

There are other embedders such as
1. OllamaDocumentEmbedder
2. OllamaTextEmbedder
3. OpenAIDocumentEmbedder
4. OpenAITextEmbedder
5. SentenceTransformersTextEmbedder
6. SentenceTransformersDocumentEmbedder

https://docs.haystack.deepset.ai/v2.0/docs/embedders

In [35]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

### Write Documents to the DocumentStore

In [36]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches: 100%|██████████| 5/5 [00:00<00:00, 25.20it/s]


151

### Building the RAG pipeline

##### -> Initialize a Text Embedder
We need a text embedder to create embedding for the user's question. This created embedding will later be used by the retriever to retrieve relevant documents from the DocumentStore.

You must use the same mode you used to generate embeddings for the document with this as well.

In [37]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

##### -> Initialize the Retriever
The retrieve makes use of the document store we initialized earlier.

In [38]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

##### -> Define a Template Prompt

Use the Jinja2 syntax inside the prompt string

In [39]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

##### -> Initialize a Generator
Generators are the components that interact with Large Language Models (LLMs).

The generator depends on the model you plan to use:

- MistralChatGenerator
- OllamaChatGenerator
- OllamaGenerator
- OpenAIChatGenerator
- OpenAIGenerator
- VertexAICodeGenerator
- AnthropicGenerator
- AnthropicChatGenerator

In [40]:
from dotenv import load_dotenv
load_dotenv()

import os
from haystack.components.generators import OpenAIGenerator


if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("OPENAI_API_KEY is not available!")

generator = OpenAIGenerator(model="gpt-3.5-turbo")

##### -> Build the Pipeline
Add all the components to a pipeline and connect them.

Connectinos from text_embedder -> query_embedding -> retriever -> prompt_builder -> llm

NOTE: You must know the name of the input and output of each pipeline component

In [41]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()

# Add components to the pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

##### -> Asking a Question

In [42]:
question = "What does Rhodes Statue look like?"
question = "Why did people visit the Temple of Artemis?"
question = "What happened to the Tomb of Mausolus?"
question = "Why did people build Great Pyramid of Giza?"

response = basic_rag_pipeline.run({ "text_embedder": {"text": question}, "prompt_builder": {"question": question} })

print(response["llm"]["replies"][0])

INFO - haystack.core.pipeline.base -  Warming up component text_embedder...
INFO - haystack.core.pipeline.pipeline -  Running component text_embedder
Batches: 100%|██████████| 1/1 [00:00<00:00, 56.60it/s]
INFO - haystack.core.pipeline.pipeline -  Running component retriever
INFO - haystack.core.pipeline.pipeline -  Running component prompt_builder
INFO - haystack.core.pipeline.pipeline -  Running component llm


The Great Pyramid of Giza was built as a tomb for Fourth Dynasty pharaoh Khufu.
